In [5]:
import cv2
import os
import matplotlib.pyplot as plt

def draw_yolo_annotations(image_path, label_path, class_names, output_path):
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return

    # Get image dimensions
    height, width, _ = image.shape

    # Read the YOLO annotation file
    if not os.path.exists(label_path):
        print(f"Error: Annotation file {label_path} not found")
        return

    with open(label_path, 'r') as file:
        lines = file.readlines()

    for line in lines:
        parts = line.strip().split()
        class_id = int(parts[0])
        x_center, y_center, w, h = map(float, parts[1:])

        # Convert YOLO format (relative) to absolute coordinates
        x1 = int((x_center - w / 2) * width)
        y1 = int((y_center - h / 2) * height)
        x2 = int((x_center + w / 2) * width)
        y2 = int((y_center + h / 2) * height)

        # Draw the bounding box
        color = (0, 255, 0)  # Green color for boxes
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Put label text
        label = class_names[class_id] if class_id < len(class_names) else str(class_id)
        cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Display the image
    cv2.imwrite(output_path, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


In [9]:

# Example usage
image_path = 'datasets/video/train/images/frame_0000_jpg.rf.6abcf8b28e82635113b3c9ea82814ccb.jpg'
label_path = 'datasets/video/train/labels/frame_0000_jpg.rf.6abcf8b28e82635113b3c9ea82814ccb.txt'
class_names = ['player']  # Modify with actual class names
output_path = "sample3.jpg"

draw_yolo_annotations(image_path, label_path, class_names, output_path)


In [1]:
def visualize_yolo_detections(image_path, detections, class_names, output_path):
    """
    Draw YOLO detections on the image and save it.
    
    Parameters:
        image_path (str): Path to the image.
        detections (list): List of detections in format [x1, y1, x2, y2, confidence, class_id].
        class_names (list): List of class labels.
        output_path (str): Path to save the annotated image.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return

    for detection in detections:
        x1, y1, x2, y2, confidence, class_id = detection

        # Choose a color for each class
        color = (0, 255, 0)  # Green
        label = f"{class_names[class_id]}: {confidence:.2f}"

        # Draw bounding box
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Put label text above the box
        cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Save the output image
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")



In [9]:
from ultralytics import YOLO
import cv2
import os 

model = YOLO("runs/detect/train3/weights/best.pt")


for image_path in os.listdir("datasets/player_tracking/test/images/"):
    # Run inference
    results = model("datasets/player_tracking/test/images/" + image_path)

    # Extract detections
    detections = []
    for result in results:
        for box in result.boxes.data:
            x1, y1, x2, y2, conf, class_id = box.tolist()
            detections.append([int(x1), int(y1), int(x2), int(y2), conf, int(class_id)])

    # Class names from YOLO model
    class_names = model.names

    # Visualize detections
    visualize_yolo_detections("datasets/player_tracking/test/images/" + image_path, 
                              detections, class_names, 
                              "outputs/detection/" + image_path)



image 1/1 /home/dangnh/b3/cv/datasets/player_tracking/test/images/313_jpg.rf.d407552e0ed086d0601fd0463f2ab979.jpg: 640x640 10 players, 5.2ms
Speed: 1.4ms preprocess, 5.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Annotated image saved to outputs/detection/313_jpg.rf.d407552e0ed086d0601fd0463f2ab979.jpg

image 1/1 /home/dangnh/b3/cv/datasets/player_tracking/test/images/686_jpg.rf.6f048403508f08b18dfc6ca3c2c99ff1.jpg: 640x640 11 players, 5.2ms
Speed: 1.0ms preprocess, 5.2ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)
Annotated image saved to outputs/detection/686_jpg.rf.6f048403508f08b18dfc6ca3c2c99ff1.jpg

image 1/1 /home/dangnh/b3/cv/datasets/player_tracking/test/images/868_jpg.rf.29fae6be16ed3a5a11ad4c90db4b5ec8.jpg: 640x640 8 players, 5.2ms
Speed: 1.1ms preprocess, 5.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Annotated image saved to outputs/detection/868_jpg.rf.29fae6be16ed3a5a11ad4c90db4b5ec8.jpg

image 1/1 /ho